In [1]:
import pathlib
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi as ca
from SE3Lie import se3, SE3, se3_diff_correction_inv, dot_plot_draw
from casadi.tools.graph import graph, dotdraw, dotsave
import os
import pydot
from PIL import Image

In [2]:
r = ca.DM([0.1, 0.2, 0.3])
v = ca.DM([0.1, 0.2, 0.3])
R = Dcm.from_euler(ca.DM([0.1, 0.2, 0.3]))
inp = ca.horzcat(np.transpose(r), np.transpose(v))
veeSe3 = np.transpose(inp)
uInv = se3_diff_correction_inv(veeSe3)

In [3]:
x = ca.SX.sym("x")
y = ca.SX.sym("y")
z = ca.SX.sym("z")
theta_1 = ca.SX.sym("theta_1")
theta_2 = ca.SX.sym("theta_2")
theta_3 = ca.SX.sym("theta_3")
veeSym = [x, y, z, theta_1, theta_2, theta_3]
# veeSym = ca.vertcat(x,y,z,theta1,theta2,theta3)
sym_uInv_old = se3_diff_correction_inv(veeSym)
# symUinv = se3_diff_correction_inv(ca.SX[x,y,z,theta1,theta2,theta3])
display(sym_uInv_old)

SX(@1=1, @2=((sq(theta_1)+sq(theta_2))+sq(theta_3)), @3=sqrt(@2), @4=(@1-(cos(@3)/@2)), @5=(sin(@3)/@3), 
[[(@1-(@4*(sq(theta_3)+(theta_1*theta_2)))), ((@4*sq(theta_1))-(@5*theta_3)), ((@5*theta_1)+(@4*(theta_3*theta_1))), (-(@4*((((theta_3*z)+(theta_1*y))+(z*theta_3))+(y*theta_2)))), ((@4*((theta_1*x)+(y*theta_1)))-(@5*z)), ((@5*y)+(@4*((theta_3*x)+(z*theta_1))))], 
 [((@5*theta_3)+(@4*(theta_1*theta_2))), (@1-(@4*(sq(theta_3)+sq(theta_1)))), ((@4*(theta_3*theta_1))-(@5*theta_1)), ((@5*z)+(@4*((theta_1*y)+(x*theta_2)))), (-(@4*((((theta_3*z)+(theta_1*x))+(z*theta_3))+(x*theta_1)))), ((@4*((theta_3*y)+(z*theta_2)))-(@5*x))], 
 [((@4*(theta_1*theta_3))-(@5*theta_2)), ((@5*theta_1)+(@4*(theta_2*theta_3))), (@1-(@4*((theta_2*theta_1)+sq(theta_1)))), ((@4*((theta_1*z)+(x*theta_3)))-(@5*y)), ((@5*x)+(@4*((theta_2*z)+(y*theta_3)))), (-(@4*((((theta_2*y)+(theta_1*x))+(y*theta_2))+(x*theta_1))))], 
 [00, 00, 00, (@1-(@4*(sq(theta_3)+sq(theta_2)))), ((@4*(theta_2*theta_1))-(@5*theta_3)), ((@5*t

In [4]:
ca.SX_eye(6)

SX(@1=1, 
[[@1, 00, 00, 00, 00, 00], 
 [00, @1, 00, 00, 00, 00], 
 [00, 00, @1, 00, 00, 00], 
 [00, 00, 00, @1, 00, 00], 
 [00, 00, 00, 00, @1, 00], 
 [00, 00, 00, 00, 00, @1]])

In [5]:
# TEST Computing UInv from scratch using casadi


def se3_diff_correction_inv_test(v):  # U_inv of se3 input vee operator
    # v = se3.vee(v)  #This only applies if v is inputed from Lie Group format

    v_so3 = v[
        3:6
    ]  # grab only rotation terms for so3 uses ## changed to match NASAULI paper order of vee v[3:6]
    X_so3 = so3.wedge(v_so3)  # wedge operator for so3
    theta = ca.norm_2(
        so3.vee(X_so3)
    )  # theta term using norm for sqrt(theta1**2+theta2**2+theta3**2)

    if type(v[1]) == "casadi.casadi.SX":
        c1 = ca.sin(theta) / theta
        c2 = 1 - ca.cos(theta) / theta**2
    elif type(v[1]) == "int" and theta < eps:
        c1 = 1 - theta**2 / 6 + theta**4 / 120
        c2 = 0.5 - theta**2 / 24 + theta**4 / 720
    else:
        c1 = ca.sin(theta) / theta
        c2 = 1 - ca.cos(theta) / theta**2

    ad = se3.ad_matrix(v)
    I = ca.SX_eye(6)
    u_inv = ca.SX(6, 6)
    u_inv = I + c1 * ad + c2 * se3.matmul(ad, ad)
    return u_inv

In [6]:
new_uinv = se3_diff_correction_inv_test(veeSym)
new_uinv

SX(@1=1, @2=((sq(theta_1)+sq(theta_2))+sq(theta_3)), @3=sqrt(@2), @4=(@1-(cos(@3)/@2)), @5=(sin(@3)/@3), 
[[(@1-(@4*(sq(theta_3)+(theta_1*theta_2)))), ((@4*sq(theta_1))-(@5*theta_3)), ((@5*theta_1)+(@4*(theta_3*theta_1))), (-(@4*((((theta_3*z)+(theta_1*y))+(z*theta_3))+(y*theta_2)))), ((@4*((theta_1*x)+(y*theta_1)))-(@5*z)), ((@5*y)+(@4*((theta_3*x)+(z*theta_1))))], 
 [((@5*theta_3)+(@4*(theta_1*theta_2))), (@1-(@4*(sq(theta_3)+sq(theta_1)))), ((@4*(theta_3*theta_1))-(@5*theta_1)), ((@5*z)+(@4*((theta_1*y)+(x*theta_2)))), (-(@4*((((theta_3*z)+(theta_1*x))+(z*theta_3))+(x*theta_1)))), ((@4*((theta_3*y)+(z*theta_2)))-(@5*x))], 
 [((@4*(theta_1*theta_3))-(@5*theta_2)), ((@5*theta_1)+(@4*(theta_2*theta_3))), (@1-(@4*((theta_2*theta_1)+sq(theta_1)))), ((@4*((theta_1*z)+(x*theta_3)))-(@5*y)), ((@5*x)+(@4*((theta_2*z)+(y*theta_3)))), (-(@4*((((theta_2*y)+(theta_1*x))+(y*theta_2))+(x*theta_1))))], 
 [00, 00, 00, (@1-(@4*(sq(theta_3)+sq(theta_2)))), ((@4*(theta_2*theta_1))-(@5*theta_3)), ((@5*t

In [7]:
dot_plot_draw(new_uinv)